In [260]:
import numpy as np
from utils import *
from tabulate import tabulate

In [261]:
%run utils.py
class Channel:
	def __init__(self, G, H) -> None:
		self.n = 4
		self.k = 2
		self.d = self.n - self.k
		self.G = G
		self.H = H
		self.codeword = get_codeword(self.G, code=generate_binary(self.k))
		self.std_arr = get_standard_array(n = self.n, codeword = self.codeword)
		# print("Codewords:", self.codeword)
		# print("Standard Array", self.std_arr)

		if self.G.shape != (self.n, self.k) or self.H.shape != (self.d, self.n):
			raise ValueError("Matrix shape mismatch")
		
		
	def encode(self, data:str) -> list:
		if len(data) != self.k:
			raise ValueError
		numpy_equiv_data = np.array(list(data), dtype=int)
		codeword = np.matmul(self.G, numpy_equiv_data)%2
		return list(codeword)

	def add_error(self, p:float, codeword) -> list:
		if p < 0 and p > 1:
			raise ValueError("Expected between 0 and 1")
		error = bsc(p)
		y_p = np.bitwise_xor(codeword, error)
		
		return y_p, error
		

	def decode(self, y_prime:list, debug:bool = False) -> str:
		coset_synd = {} ## dict of coset leader and syndrome
		for arr in self.std_arr.values():
			coset_synd[arr[0]] = np.matmul(self.H,convert_str_to_numpy(arr[0]))%2
		
		syndrome = np.matmul(self.H, y_prime)%2
		
		coset_leader = get_key(coset_synd, syndrome)
		
		y = np.bitwise_xor(y_prime, convert_str_to_numpy(coset_leader))
		if debug: 
			print("Coset and Syndrome", coset_synd)
			print(f"Syndrome: {syndrome}")
			print(f'coset leader: {coset_leader}')
		return string_equiv_from_list(y[0:2])
		
		


In [262]:
%run utils.py
def simulate(channel:Channel, n:int, p:float, debug:bool = False):
    X = generate_binary(2)
    word_error_count = 0
    ser = 0
    for i in range(n):
        x = string_equiv_from_list(random.choice(X))  ## randomly choose a 2 bit binary 

        codeword = channel.encode(x)
        
        y_prime, er = channel.add_error(p=p, codeword=codeword)
        
        x_prime = channel.decode(y_prime=y_prime, debug=debug)
          
        if debug:
            print(f"x: {x}")
            print(f"codeword: {codeword}")
            print(f"error: {er}")
            print(f"yprime: {y_prime}")
            print(f"x_prime: {x_prime}")
            print("-------------------------------------")
        
        ## calculate ser
        temp_ser_count = 0
    
        for (x_dash_i, x_i) in zip(x_prime,x):
            if x_dash_i != x_i:
                temp_ser_count+=1
        
        ser+=(temp_ser_count/2)

        ## calculate wer
        if temp_ser_count > 0:
            word_error_count += 1
            
    # print(word_error_count, ser/n)
    return word_error_count/n, ser/n

In [265]:
def main():
	G = np.array([  [1,0],
                [0,1],
                [1,0],
                [1,1]
              ])

	H = np.array([  [1,0,1,0],
					[1,1,0,1]
				])

	c = Channel(G, H)

	num_simulations = [10, 100, 1000, 10000]
	output = []
	for i in num_simulations:
		wer, ser = simulate(channel = c, n = i, p = 0.3, debug=False)
		output.append([i, wer, ser])
	print(tabulate(output, headers= ['Simulation', 'Wer', 'Ser'], tablefmt="pretty", numalign='center'))
	

In [266]:
main()

+------------+--------+---------+
| Simulation |  Wer   |   Ser   |
+------------+--------+---------+
|     10     |  0.5   |   0.3   |
|    100     |  0.43  |  0.27   |
|    1000    | 0.464  | 0.2905  |
|   10000    | 0.4447 | 0.28085 |
+------------+--------+---------+


In [267]:
def theoretic_limit(p):
	p = p
	q = 1-p
	s = 0
	ser = 8*(p**2)*(q**2) + p*(q**3) + 5*(p**3)*q + 2*p**4
	wer =  3 * p * q**3  + q**4
	print(ser/2, 1 - wer)
	

theoretic_limit(0.3)

0.28319999999999995 0.45120000000000016
